In [77]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import Search
from folium.plugins import MarkerCluster
import json
import os

In [78]:
redline_df = pd.read_csv('C:\Files\Stations Red.csv')
redline_df2 = pd.read_csv('C:\Files\Stations Red2.csv')
greenline_df = pd.read_csv('C:\Files\Stations Green.csv')
blueline_df = pd.read_csv('C:\Files\Blueline tentative.csv')
blueline_df2 = pd.read_csv('C:\Files\Blueline tentative2.csv')
blueline_df2.columns = ['Station Name'] + blueline_df2.columns[1:].tolist()
df = pd.read_csv('C:\Files\dataframeAreasWithALLCoord.csv')
df.columns = ['Place', 'Latitude', 'Longitude']

In [79]:
new_values = ['Jabal Ali Metro Station', 25.057811, 55.126961]

# Replace the first row with new values
redline_df2.iloc[0] = new_values

In [80]:
# Define functions
def add_marker(cluster, df, place_type, color, icon):
    for index, row in df.iterrows():
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            tooltip=row['Place'],
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
            ).add_to(cluster)

def add_circle_markers(map_obj, df, color):
    for index, row in df.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            tooltip=row['Station Name']
        ).add_to(map_obj)

def add_polylines(map_obj, df, color, weight=4, opacity=1.0, dash_array=None, tooltip=None):
    coordinates = df[['Latitude', 'Longitude']].values.tolist()
    folium.PolyLine(
        locations=coordinates,
        color=color,
        weight=weight,
        opacity=opacity,
        dash_array=dash_array,
        tooltip=tooltip
    ).add_to(map_obj)


def add_geojson(map_obj, path, name, color, weight=2, fill_color='none', fill_opacity=0.5):
    with open(path, encoding='utf-8') as f:
        data = json.load(f)
    
    if 'features' in data:
        # It's a FeatureCollection
        filtered_features = [feature for feature in data['features'] if feature['geometry']['type'] in ['Polygon', 'MultiPolygon']]
        filtered_geojson = {
            "type": "FeatureCollection",
            "features": filtered_features
        }
    else:
        # Handle other types of GeoJSON
        if data['type'] in ['Polygon', 'MultiPolygon']:
            filtered_geojson = data
        else:
            return  # Skip if it's not a Polygon or MultiPolygon

    folium.GeoJson(filtered_geojson, name=name, style_function=lambda feature: {
        'color': color,
        'weight': weight,
        'fillColor': fill_color,
        'fillOpacity': fill_opacity
    
    }).add_to(map_obj)

# Paths
geojson_path = r'C:\Files\dubai border updated.geojson'

# Map initialization
map = folium.Map(location=[25.276987, 55.296249], tiles='OpenStreetMap.DE', zoom_start=10)

# Marker Cluster
cluster = MarkerCluster(name='Cluster Optimization', options={
    'maxClusterRadius': 80,
    'disableClusteringAtZoom': 15,
    'spiderfyOnMaxZoom': True,
}).add_to(map)

# Add markers
add_marker(cluster, df, 'Area', 'lightblue', 'circle')

# Add CircleMarkers directly to the map
add_circle_markers(map, redline_df, 'red')
add_circle_markers(map, redline_df2, 'red')
add_circle_markers(map, greenline_df, 'green')
add_circle_markers(map, blueline_df, 'blue')
add_circle_markers(map, blueline_df2, 'blue')

# Add Polylines directly to the map
add_polylines(map, redline_df, 'red', tooltip='Red Line')
add_polylines(map, redline_df2, 'red', tooltip='Red Line')
add_polylines(map, greenline_df, 'green', tooltip='Green Line')
add_polylines(map, blueline_df, 'blue', opacity=0.7, dash_array='10,5', tooltip='Tentative Blue Line')
add_polylines(map, blueline_df2, 'blue', opacity=0.7, dash_array='10,5', tooltip='Tentative Blue Line')

# Add GeoJSON layers with tooltips
add_geojson(map, r'C:\Files\Upcoming.geojson', 'Upcoming Developments', 'black', weight=2, fill_color='red', fill_opacity=0.2)

# Add TileLayer and LayerControl
folium.TileLayer('Esri.WorldImagery').add_to(map)
folium.TileLayer('Cartodb dark_matter').add_to(map)
folium.TileLayer('Cartodb Positron').add_to(map)

folium.LayerControl().add_to(map)

#map.save('C:\\Files\\Spatial Analysis Dubai.html')
map
